In [69]:
%load_ext autoreload
%autoreload 2
import IPython.display
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pickle as pkl
import os
import json
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
! git clone https://github.com/CSAILVision/ADE20K.git

Cloning into 'ADE20K'...


In [7]:
# Load index with global information about ADE20K
DATASET_PATH = './ADE20K_2021_17_01/'
index_file = 'index_ade20k.pkl'
with open('{}/{}'.format(DATASET_PATH, index_file), 'rb') as f:
    index_ade20k = pkl.load(f)

In [9]:
index_ade20k.keys()

dict_keys(['filename', 'folder', 'objectIsPart', 'objectPresence', 'objectcounts', 'objectnames', 'proportionClassIsPart', 'scene', 'wordnet_found', 'wordnet_level1', 'wordnet_synset', 'wordnet_hypernym', 'wordnet_gloss', 'wordnet_frequency', 'description'])

In [27]:
len(index_ade20k["filename"])

27574

In [10]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [30]:
dirName = './ADE20K_2021_17_01/images/ADE/validation'
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)

In [31]:
listOfFiles

['./ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025\\instance_000_ADE_val_00000025.png',
 './ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025\\instance_001_ADE_val_00000025.png',
 './ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025\\instance_002_ADE_val_00000025.png',
 './ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025\\instance_003_ADE_val_00000025.png',
 './ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025\\instance_004_ADE_val_00000025.png',
 './ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025\\instance_005_ADE_val_00000025.png',
 './ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025\\instance_006_ADE_val_00000025.png',
 './ADE20K_2021_17_01/images/ADE/validation\\cultural\\apse__indoor\\ADE_val_00000025.jpg',
 './ADE20K_2021_17_01/images/ADE/validation\\cu

In [40]:
import shutil

for file in listOfFiles:
    if file.endswith(".jpg"):
        #shutil.move(file, "./img/"+file[-22:]) # train
        shutil.move(file, "./img/"+file[-20:]) # validation
    if file.endswith(".json"):
        #shutil.move(file, "./annotations/"+file[-23:]) # train
        shutil.move(file, "./annotations/"+file[-21:]) # validation

In [42]:
len(os.listdir("./annotations"))

27574

In [44]:
os.listdir("./annotations")[0]

'ADE_frame_00000001.json'

In [49]:
f = open("./annotations/"+os.listdir("./annotations")[1])
data = json.load(f)

In [58]:
data.keys()

dict_keys(['annotation'])

In [63]:
list_instances = []
for file in os.listdir("./annotations"):
    f = open("./annotations/"+file)
    ann = json.load(f)["annotation"]
    filename = ann["filename"]
    for instance in ann["object"]:
        instance["filename"] = filename
        list_instances.append(instance)

In [64]:
len(list_instances)

707868

In [66]:
f = open('annotations.json', 'w')
json.dump(list_instances, f)

In [81]:
f = open("./annotations.json")
data = json.load(f)
df_ade = pd.DataFrame(data)
df_ade

,id,name,name_ndx,hypernym,raw_name,attributes,depth_ordering_rank,occluded,crop,parts,instance_mask,polygon,saved_date,filename
0,0,sea,2264,"[sea, body of water, water, thing, physical en...",sea water,[],1,no,0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_frame_00000001/instance_000_ADE_frame_0000...,"{'x': [0, 0, 197, 234, 275, 342, 381, 426, 452...",26-May-2016 15:52:18,ADE_frame_00000001.jpg
1,1,track,2830,[],track,[],2,no,0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_frame_00000001/instance_001_ADE_frame_0000...,"{'x': [500, 499, 201, 188, 1, 1, 82, 115], 'y'...",26-May-2016 15:54:27,ADE_frame_00000001.jpg
2,2,sand,2212,"[sand, soil, dirt, earth, ground, material, st...",sand beach,[],3,no,0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_frame_00000001/instance_002_ADE_frame_0000...,"{'x': [499, 499, 463, 447, 421, 398, 359, 306,...",26-May-2016 15:55:11,ADE_frame_00000001.jpg
3,3,"person, individual, someone, somebody, mortal,...",1831,"[person, individual, someone, somebody, mortal...",person,walking,4,no,0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_frame_00000001/instance_003_ADE_frame_0000...,"{'x': [350, 350, 346, 345, 344, 347, 346, 346,...",26-May-2016 15:56:11,ADE_frame_00000001.jpg
4,4,"backpack, back pack, knapsack, packsack, rucks...",90,"[backpack, back pack, knapsack, packsack, ruck...",backpack,[],5,no,0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_frame_00000001/instance_004_ADE_frame_0000...,"{'x': [354, 356, 357, 354, 351, 351, 349, 349,...",26-May-2016 15:56:24,ADE_frame_00000001.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707863,0,ring,2123,[],ring,,1,[],0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_val_00002000/instance_000_ADE_val_00002000...,"{'x': [239, 237, 298, 297, 300, 295, 184, 296,...",18-May-2009 10:00:53,ADE_val_00002000.jpg
707864,1,"stairs, steps",2530,"[stairs, steps, stairway, staircase, way, arti...",steps,,2,[],0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_val_00002000/instance_001_ADE_val_00002000...,"{'x': [162, 189, 185, 158, 153, 148, 148, 143,...",18-May-2009 10:00:53,ADE_val_00002000.jpg
707865,2,platform,1924,"[platform, horizontal surface, level, surface,...",platform,,3,no,0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_val_00002000/instance_002_ADE_val_00002000...,"{'x': [50, 51, 78, 80, 225, 224, 232, 241, 237...",18-May-2009 10:00:53,ADE_val_00002000.jpg
707866,3,wall,2978,"[wall, partition, divider, structure, construc...",wall,,4,[],0,"{'hasparts': [], 'ispartof': [], 'part_level': 0}",ADE_val_00002000/instance_003_ADE_val_00002000...,"{'x': [50, 172, 175, 175, 175, 85, 84, 77, 76,...",18-May-2009 10:00:53,ADE_val_00002000.jpg


In [85]:
len(set(df_ade["name"]))

3550

In [97]:
df_ade["count"]=1
df = df_ade.groupby(['raw_name']).agg({'count': 'sum'})
df

,count
raw_name,
",pimtaom",1
-,1
1,2
_ñamts,1
abacus,1
...,...
zebra,1
ziggurat,1
zipper,1


In [99]:
df[df["count"]>=100]

,count
raw_name,
aerial,205
air conditioning,640
air vent,117
airplane,386
animal,611
...,...
window,53200
windows,470
windshield,1076


In [84]:
df_ade[df_ade["raw_name"]=="1"]

,id,name,name_ndx,hypernym,raw_name,attributes,depth_ordering_rank,occluded,crop,parts,instance_mask,polygon,saved_date,filename
591797,2,1,3379,[],1,[],3,yes,1,"{'hasparts': 3, 'ispartof': [], 'part_level': 0}",ADE_train_00022269/instance_002_ADE_train_0002...,"{'x': [161, 92, 92, 161], 'y': [119, 119, 58, ...",20-Oct-2016 12:38:46,ADE_train_00022269.jpg
591798,3,1,3379,[],1,11,4,yes,1,"{'ispartof': 2, 'hasparts': [], 'part_level': 1}",ADE_train_00022269/instance_003_ADE_train_0002...,"{'x': [152, 94, 94, 152], 'y': [107, 107, 58, ...",20-Oct-2016 12:39:01,ADE_train_00022269.jpg
